In [1]:
import argparse
import os
import numpy as np
import torch
import cv2, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *

from dataloaders.utils import decode_segmap
MI = np.load('meanimage.npy')

numClass={'pascal':6,
'coco':21,
'cityscapes':19}
classes = ['drusen', 'hemorrhage', 'exudate', 'scar', 'others']
for cid, c in enumerate(classes):
    if not os.path.isdir('result/'+c):
        os.mkdir('result/' + c)

# parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")

# parser.add_argument('--backbone', type=str, default='resnet',choices=['resnet', 'xception', 'drn', 'mobilenet'],help='backbone name (default: resnet)')
# parser.add_argument('--out-stride', type=int, default=16,help='network output stride (default: 8)')
# parser.add_argument('--dataset', type=str, default='cityscapes', choices=['pascal', 'coco', 'cityscapes'],help='dataset name (default: pascal)')
# parser.add_argument('--sync-bn', type=bool, default=None,help='whether to use sync bn (default: auto)')
# parser.add_argument('--freeze-bn', type=bool, default=False,help='whether to freeze bn parameters (default: False)')
# parser.add_argument('--weightPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--imgPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--outPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
# args = parser.parse_args()
cuda = torch.cuda.is_available()
cuda = False
nclass = numClass['pascal']
model = DeepLab(num_classes=nclass, backbone='resnet', output_stride=16, sync_bn=None, freeze_bn=False)
weight_dict=torch.load('run/pascal/task4/model_best.pth.tar', map_location='cpu')
if cuda:
    model = torch.nn.DataParallel(model, device_ids=[0])
    patch_replication_callback(model)
    model = model.cuda()
    
    model.module.load_state_dict(weight_dict['state_dict'])
else:
    model.load_state_dict(weight_dict['state_dict'])
model.eval()


filenames = glob.glob('Validation-400-images/*.jpg')
kernel = np.ones((5,5),np.uint8)
kernel2 = np.ones((7,7),np.uint8)
for imgPath in filenames:
    fn = os.path.basename(imgPath)[:-4]
    outPath = 'result/' + fn +'.png'

    image = cv2.imread(imgPath)
    oriDim = image.shape
    image = cv2.resize(image, dsize=(513,513)) - MI
    image = image.astype(np.float32) / 255.
    image = image[:, :, ::-1]
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    
    
    for i in range(3):
        image[:, :, i] = image[:, :, i] - means[i]
        image[:, :, i] = image[:, :, i] / stds[i]

    image = torch.from_numpy(image.transpose((2, 0, 1)).astype(np.float32)).float().unsqueeze(0)

    if cuda:
        image = image.cuda()
        
    with torch.no_grad():
        output = model(image)
        output = output.data.cpu().numpy()
        prediction = np.argmax(output, axis=1)[0]
        ps=[]
        for cid, c in enumerate(classes):
            mask = np.zeros((prediction.shape[0], prediction.shape[1]), np.uint8) +255
            mask[prediction == cid+1] = 0
            mask = cv2.morphologyEx(255-mask, cv2.MORPH_OPEN, kernel)
            mask = 255-cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel2)

            mask = cv2.resize(mask,dsize=(oriDim[1],oriDim[0]), interpolation=cv2.INTER_NEAREST)
            ps.append(np.mean(prediction == cid+1))
            cv2.imwrite('result/' + c + '/' + fn+'.png', mask)

        segmap = decode_segmap(prediction, dataset='pascal')
        segmap = (segmap*255).astype(np.uint8)
        segmap = cv2.resize(segmap,dsize=(oriDim[1],oriDim[0]))
        segmap = segmap[:, :, ::-1]
        cv2.imwrite(outPath, segmap)
    print('Done inference '+fn,'percentage:', ps)
exit(1)

Done inference V0366 percentage: [6.07974343482705e-05, 0.0, 0.0, 0.0, 0.0]
Done inference V0213 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0333 percentage: [0.13576067089968805, 0.0, 0.0, 0.0, 0.0]
Done inference V0108 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0264 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0323 percentage: [0.0, 0.0, 0.0, 0.04111426497801793, 0.0]
Done inference V0347 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0064 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0389 percentage: [0.0, 0.0, 0.003666845259130065, 0.003412256002796682, 0.0]
Done inference V0381 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0306 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0307 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0188 percentage: [0.00014439390657714244, 0.0, 0.0, 0.0, 0.0]
Done inference V0014 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0296 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inferenc

Done inference V0332 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0230 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0059 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0278 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0134 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0327 percentage: [0.0002963874924478187, 0.0, 0.0, 0.0, 0.0]
Done inference V0219 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0353 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0216 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0076 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0226 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0001 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0394 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0175 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0273 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0075 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0065 percentage: [0.0,

Done inference V0140 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0161 percentage: [0.0, 0.0, 0.022126466263123697, 0.0010297565442738317, 0.0]
Done inference V0274 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0004 percentage: [0.0, 0.0, 0.0, 0.0, 0.00945780088080283]
Done inference V0035 percentage: [0.0, 0.0, 0.002401498656756685, 0.0, 0.0]
Done inference V0118 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0202 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0029 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0268 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0235 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0350 percentage: [0.07269473228229768, 0.0, 0.0, 0.0, 0.0]
Done inference V0113 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0287 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0283 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0131 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0211 percentage

Done inference V0386 percentage: [0.0017897244736272129, 0.0, 0.0, 0.0, 0.0]
Done inference V0259 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0368 percentage: [0.003761841250299237, 0.0, 0.025268933650999928, 0.0, 0.0]
Done inference V0310 percentage: [0.005912550490369307, 0.0, 0.0, 0.0, 0.0]
Done inference V0040 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0196 percentage: [0.0, 0.0, 0.015047365001196949, 0.0, 0.0]
Done inference V0214 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0190 percentage: [0.0, 0.0014819374622390936, 0.045837465658949195, 0.18609334686076248, 0.0]
Done inference V0099 percentage: [0.0, 0.0, 0.23768376974491676, 0.0, 0.0]
Done inference V0291 percentage: [0.0, 0.010149371696514408, 0.0, 0.0, 0.0]
Done inference V0270 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0063 percentage: [0.003324859690921043, 0.0, 0.0, 0.0, 0.0]
Done inference V0308 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0129 percentage: [0.0, 0.0, 0.